In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
%%bash
(cd ../data/; ls) > all_dirs.txt
rm all_files.txt
touch all_files.txt
rm data_fullnesses.txt

In [3]:
%%bash
(cd ../data/'Agriculture & Rural Development'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Aid Effectiveness'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Economy & Growth'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Education'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Energy & Mining'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Environment'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'External Debt'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Financial Sector'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Gender'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Health'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Infrastructure'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Poverty'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Private Sector'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Public Sector'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Science & Technology'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Social Development'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Social Protection & Labor'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Trade'/; ls) >> all_files.txt; echo end >> all_files.txt
(cd ../data/'Urban Development'/; ls) >> all_files.txt; echo end >> all_files.txt

In [4]:
list_of_dirs = ['Agriculture & Rural Development',
                'Aid Effectiveness',
                'Economy & Growth',
                'Education',
                'Energy & Mining',
                'Environment',
                'External Debt',
                'Financial Sector',
                'Gender',
                'Health',
                'Infrastructure',
                'Poverty',
                'Private Sector',
                'Public Sector',
                'Science & Technology',
                'Social Development',
                'Social Protection & Labor',
                'Trade',
                'Urban Development']

In [5]:
def read_list_from_input(file):
    """
    В файле хранятся имена всех таблиц
    После файлов из каждой папки стоит 'end'
    Функция возвращает один блок - файлы из одной папки
    """
    
    ret = []
    for line in file:
        if line == '\n':
            continue
        if line == 'end\n':
            break
        ret.append(line[:-1:])
    return ret

In [6]:
def find_eu_and_us_indices(df):
    """
    Все просто! Функция ищет в Dataframe - df
    Строки относящиеся к Европе и Штатам
    """
    
    eu = 'European Union'
    us = 'United States'
    for i, name in enumerate(df['Country Name']):
        if name == eu:
            eui = i
        if name == us:
            usi = i
    return (eui, usi)

In [7]:
fpath = '../data/'
all_files = 'all_files.txt'
data_fullnesses = 'data_fullnesses.txt'
f = open(all_files, 'r')
w = open(data_fullnesses, 'w')

In [8]:
%%time
# select from all directories  
for i, directory in enumerate(list_of_dirs):
    list_of_files = read_list_from_input(f)
    # select from all files
    for j, csv in enumerate(list_of_files):
        # read csv
        cur_file_path = fpath + directory + '/' + csv
        data = pd.read_csv(cur_file_path, skiprows=[0, 2])
        
        # take info about Eu and Us
        eui, usi = find_eu_and_us_indices(data)
        data_eu_and_us = pd.DataFrame({'Eu': data.iloc[eui][4:], 'Us': data.iloc[usi][4:]})
        # how much data is not null
        counts = data_eu_and_us.describe().T['count']
        eu_count = counts[0] / data_eu_and_us.shape[0]
        us_count = counts[1] / data_eu_and_us.shape[0]
        
        # Вот здесь можно запоминать 'хорошие' таблицы, чтобы потом соединять их
        # Сейчас надо другим заниматься, потом доделаю
        # А вообще, если Вадиму скучно, он тоже может этим заняться с:
        
        # write to data_fulnesses.txt
        output = f"EU: {eu_count:.4f} US: {us_count:.4f} : '{directory}/{csv}'\n"
        w.write(output)
    w.write('\n')
    continue

CPU times: user 6.47 s, sys: 372 ms, total: 6.84 s
Wall time: 6.99 s


In [19]:
file = open(data_fullnesses, 'r')
eu_list = []
us_list = []
eu_good_list = []
us_good_list = []
for line in file:
    if line == '\n':
        continue
    line = line.split()
    # Выбираем только заполненные >65% таблицы
    if float(line[1]) > 0.98:
        eu_good_list.append(float(line[1]))
    if float(line[3]) > 0.98:
        us_good_list.append(float(line[3]))
    # Выбираем все таблицы
    eu_list.append(float(line[1]))
    us_list.append(float(line[3])) 
    
print("Средняя заполненность всех таблиц (EU, US):\n", np.mean(eu_list), np.mean(us_list))
print("Средняя заполненность 'хороших' таблиц (EU, US):\n", np.mean(eu_good_list), np.mean(us_good_list))
print("Количество 'хороших' таблиц (EU, US):\n", len(eu_good_list), len(us_good_list))

Средняя заполненность всех таблиц (EU, US):
 0.42498059071729954 0.39494345991561175
Средняя заполненность 'хороших' таблиц (EU, US):
 0.9831000000000001 0.9831
Количество 'хороших' таблиц (EU, US):
 28 32
